In [1]:
import torch
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
#读取数据
transform = transforms.Compose(
    [transforms.Resize((32,32)),
     transforms.ToTensor()])

train_data = datasets.MNIST(root="./data", train=True, 
                            download=True,transform=transform)
test_data = datasets.MNIST(root="./data", train=False, 
                           download=True, transform=transform)

In [3]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_data, batch_size = batch_size, num_workers = 0)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, num_workers = 0)

In [4]:
import torch.nn as nn
import torch.nn.functional as F
class LeNet(nn.Module):   
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("using {} device.".format(device))
model=LeNet().to(device)

using cuda:0 device.


In [6]:
import torch.optim as optim
loss_function = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters(), lr=0.01)
epochs=10

In [7]:
import time

In [9]:
for epoch in range(epochs):
    t1 = time.perf_counter()
    train_loss = 0.0
    for data, target in train_loader:
        data,target=data.to(device),target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = loss_function(output,target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
    train_loss = train_loss / len(train_loader.dataset)
    print('Epoch:  {}  \tTraining Loss: {:.6f}'.format(epoch + 1, train_loss))
    
    print(time.perf_counter() - t1)


Epoch:  1  	Training Loss: 0.189160
16.725549900000004
Epoch:  2  	Training Loss: 0.130668
9.553532899999993
Epoch:  3  	Training Loss: 0.113287
9.607486999999992
Epoch:  4  	Training Loss: 0.117600
9.757201199999997
Epoch:  5  	Training Loss: 0.105338
9.915631699999977
Epoch:  6  	Training Loss: 0.105069
9.6895935
Epoch:  7  	Training Loss: 0.114365
9.669973900000002
Epoch:  8  	Training Loss: 0.097616
9.855310000000003
Epoch:  9  	Training Loss: 0.102663
9.434495400000003
Epoch:  10  	Training Loss: 0.097821
9.610926799999987


In [10]:
correct = 0
total = 0
with torch.no_grad():  # 训练集中不需要反向传播
    for data in test_loader:
        images, labels = data
        images, labels =images.to(device), labels.to(device)
        output = model(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

Accuracy of the network on the test images: 97 %
